### 🚀 **`Optimización de Querys`**

In [1]:
import config
import psycopg2

In [2]:
## Abro conexión
conn = psycopg2.connect(
    dbname = config.DATABASE,
    user = config.USER, 
    password = config.PASS,
    host = "localhost",
    port = "5432"
)

cursor = conn.cursor()

### `Plan de Ejecución`

#### 📌 `Query Pregunta 1.`

In [3]:
query = """
EXPLAIN ANALYZE
WITH income_credit_bins AS (
  SELECT 
    SK_ID_CURR,
    TARGET,
    NTILE(10) OVER (ORDER BY AMT_INCOME_TOTAL) AS income_decile,
    NTILE(10) OVER (ORDER BY AMT_CREDIT) AS credit_decile
  FROM application_train
)
SELECT 
  income_decile,
  credit_decile,
  COUNT(*) AS total_clients,
  SUM(TARGET) AS total_defaults,
  ROUND(SUM(TARGET) * 1.0 / COUNT(*), 4) AS default_rate
FROM income_credit_bins
GROUP BY income_decile, credit_decile
ORDER BY income_decile, credit_decile;
"""

cursor.execute(query)
plan = cursor.fetchall()
for row in plan:
    print(row[0])


Sort  (cost=112172.46..112249.33 rows=30751 width=56) (actual time=497.531..497.535 rows=100 loops=1)
  Sort Key: (ntile(10) OVER (?)), (ntile(10) OVER (?))
  Sort Method: quicksort  Memory: 29kB
  ->  HashAggregate  (cost=109188.33..109880.22 rows=30751 width=56) (actual time=497.320..497.503 rows=100 loops=1)
        Group Key: ntile(10) OVER (?), (ntile(10) OVER (?))
        Batches: 1  Memory Usage: 1569kB
        ->  WindowAgg  (cost=97656.68..103038.11 rows=307511 width=28) (actual time=451.746..478.384 rows=307511 loops=1)
              ->  Sort  (cost=97656.66..98425.44 rows=307511 width=20) (actual time=405.905..430.636 rows=307511 loops=1)
                    Sort Key: application_train.amt_income_total
                    Sort Method: external merge  Disk: 9896kB
                    ->  WindowAgg  (cost=57938.18..63319.61 rows=307511 width=20) (actual time=331.043..357.392 rows=307511 loops=1)
                          ->  Sort  (cost=57938.17..58706.94 rows=307511 width=16)

**Análisis** 

El plan de ejecución para la query que responde la pregunta 1, muestra que la query realiza un escaneo secuencial completo sobre la tabla `application_train` y utiliza ordenamientos (sort) sobre las columnas `AMT_INCOME_TOTAL` y `AMT_CREDIT` para calcular los deciles. Estos ordenamientos son costosos y utilizan disco, lo que puede afectar el rendimiento si la tabla crece.

Actualmente, no se están utilizando índices en estas columnas, lo que explica el uso del escaneo secuencial y los altos tiempos de procesamiento en los pasos de ordenamiento.

De lo anterior, se desprenden algunas recomendaciones para optimizar la consulta:

* Crear índices sobre las columnas `AMT_INCOME_TOTAL` y `AMT_CREDIT` para acelerar los procesos de ordenamiento y mejorar el rendimiento de la consulta, especialmente si la tabla es grande o la consulta se ejecuta frecuentemente.
  
* Monitorear el uso de disco y memoria durante los ordenamientos, ya que un mayor tamaño de tabla puede incrementar significativamente los recursos requeridos.

* Revisar periódicamente el plan de ejecución tras crear los índices para confirmar que el motor de base de datos los está utilizando y que el rendimiento mejora.

#### 📌 `Query Pregunta 2.`

In [4]:
query = """
EXPLAIN ANALYZE
WITH bureau_flags AS (
  SELECT 
    SK_ID_CURR,
    COUNT(*) AS total_bureau_records
  FROM bureau
  GROUP BY SK_ID_CURR
),
flags_joined AS (
  SELECT 
    a.SK_ID_CURR,
    a.TARGET,
    CASE WHEN b.total_bureau_records IS NULL THEN 0 ELSE 1 END AS has_bureau_record
  FROM application_train a
  LEFT JOIN bureau_flags b ON a.SK_ID_CURR = b.SK_ID_CURR
)
SELECT 
  has_bureau_record,
  COUNT(*) AS total_clients,
  SUM(TARGET) AS total_defaults,
  ROUND(SUM(TARGET) * 1.0 / COUNT(*), 4) AS default_rate
FROM flags_joined
GROUP BY has_bureau_record;
"""

cursor.execute(query)
plan = cursor.fetchall()
for row in plan:
    print(row[0])

Finalize GroupAggregate  (cost=98479.55..98533.72 rows=200 width=52) (actual time=472.534..473.815 rows=2 loops=1)
  Group Key: (CASE WHEN (b.total_bureau_records IS NULL) THEN 0 ELSE 1 END)
  ->  Gather Merge  (cost=98479.55..98526.22 rows=400 width=20) (actual time=472.527..473.808 rows=6 loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Sort  (cost=97479.52..97480.02 rows=200 width=20) (actual time=467.232..467.234 rows=2 loops=3)
              Sort Key: (CASE WHEN (b.total_bureau_records IS NULL) THEN 0 ELSE 1 END)
              Sort Method: quicksort  Memory: 25kB
              Worker 0:  Sort Method: quicksort  Memory: 25kB
              Worker 1:  Sort Method: quicksort  Memory: 25kB
              ->  Partial HashAggregate  (cost=97469.88..97471.88 rows=200 width=20) (actual time=467.219..467.221 rows=2 loops=3)
                    Group Key: CASE WHEN (b.total_bureau_records IS NULL) THEN 0 ELSE 1 END
                    Batches: 1  Memory Usage: 40kB


**Análisis** 

El plan de ejecución para la segunda consulta muestra que se realiza un Hash Left Join entre las tablas application_train y bureau, utilizando procesamiento en paralelo (tres procesos en total). El acceso a la tabla application_train se realiza mediante un escaneo secuencial paralelo, mientras que la tabla bureau se agrupa previamente por SK_ID_CURR usando un GroupAggregate y se accede eficientemente mediante un Index Only Scan sobre su clave primaria.

A pesar del uso de paralelismo y de un índice en bureau, la consulta sigue siendo costosa en tiempo de ejecución (alrededor de 820 ms) debido a la gran cantidad de filas procesadas y la necesidad de agrupar y unir grandes volúmenes de datos.

Para optimizar esta consulta, se podrían considerar las siguientes acciones:

* Evaluar la creación de un índice sobre `SK_ID_CURR` en application_train si se realizan muchas uniones similares, para facilitar futuros joins.
  
* Monitorear el uso de recursos (memoria y CPU) durante el procesamiento paralelo, especialmente si la base de datos se encuentra en un entorno compartido.

* Revisar la cardinalidad y el tamaño de las tablas: Si la tabla bureau crece significativamente, considera estrategias de particionamiento o ajustes en la configuración de paralelismo de PostgreSQL.

* Revisar periódicamente el plan de ejecución tras cambios en los índices o en el volumen de datos, para asegurar que el motor de base de datos sigue utilizando los caminos más eficientes.

#### 📌 `Query Pregunta 3.`

In [5]:
query = """
EXPLAIN ANALYZE
SELECT 
  NAME_CONTRACT_TYPE,
  COUNT(*) AS total_clients,
  SUM(TARGET) AS total_defaults,
  ROUND(SUM(TARGET) * 1.0 / COUNT(*), 4) AS default_rate
FROM application_train
GROUP BY NAME_CONTRACT_TYPE
ORDER BY default_rate DESC;
"""

cursor.execute(query)
plan = cursor.fetchall()
for row in plan:
    print(row[0])

Sort  (cost=24818.88..24818.88 rows=2 width=59) (actual time=21.823..22.632 rows=2 loops=1)
  Sort Key: (round((((sum(target))::numeric * 1.0) / (count(*))::numeric), 4)) DESC
  Sort Method: quicksort  Memory: 25kB
  ->  Finalize GroupAggregate  (cost=24818.32..24818.87 rows=2 width=59) (actual time=21.815..22.626 rows=2 loops=1)
        Group Key: name_contract_type
        ->  Gather Merge  (cost=24818.32..24818.79 rows=4 width=27) (actual time=21.808..22.618 rows=6 loops=1)
              Workers Planned: 2
              Workers Launched: 2
              ->  Sort  (cost=23818.30..23818.31 rows=2 width=27) (actual time=19.289..19.289 rows=2 loops=3)
                    Sort Key: name_contract_type
                    Sort Method: quicksort  Memory: 25kB
                    Worker 0:  Sort Method: quicksort  Memory: 25kB
                    Worker 1:  Sort Method: quicksort  Memory: 25kB
                    ->  Partial HashAggregate  (cost=23818.27..23818.29 rows=2 width=27) (actual ti

**Análisis** 

El plan de ejecución para esta consulta de la pregunta 3, muestra que se realiza un escaneo secuencial paralelo sobre la tabla application_train, seguido de una agregación por el campo `NAME_CONTRACT_TYPE` utilizando procesamiento en paralelo. Posteriormente, los resultados se ordenan (Sort) por la tasa de default calculada, empleando el método quicksort en memoria.

El uso de procesamiento paralelo permite distribuir la carga entre varios trabajadores, lo que mejora el tiempo de respuesta (aproximadamente 127 ms en total). Sin embargo, el acceso a los datos sigue siendo mediante escaneo secuencial, lo que puede ser ineficiente si la tabla crece significativamente.

Para optimizar esta consulta, se podrían considerar las siguientes acciones:

* Considerar la creación de un índice sobre `NAME_CONTRACT_TYPE` si la cardinalidad de este campo aumenta o si la consulta se vuelve más compleja, aunque para agregaciones simples y pocos grupos el beneficio puede ser limitado.

* Monitorear el rendimiento a medida que crece la tabla, ya que el escaneo secuencial podría impactar los tiempos de respuesta en el futuro.

* Aprovechar el procesamiento paralelo, pero revisar la configuración de recursos del servidor para asegurar que el paralelismo no afecte otras operaciones críticas.

* Revisar periódicamente el plan de ejecución para confirmar que el motor de base de datos sigue utilizando los caminos más eficientes y ajustar los índices si cambian los patrones de consulta.

#### 📌 `Query Pregunta 4.`

In [6]:
query = """
EXPLAIN ANALYZE
WITH credit_balance AS (
  SELECT 
    SK_ID_CURR,
    AVG(AMT_BALANCE) AS avg_balance
  FROM credit_card_balance
  GROUP BY SK_ID_CURR
),
balance_bins AS (
  SELECT 
    a.SK_ID_CURR,
    a.TARGET,
    NTILE(10) OVER (ORDER BY c.avg_balance) AS balance_decile
  FROM application_train a
  LEFT JOIN credit_balance c ON a.SK_ID_CURR = c.SK_ID_CURR
)
SELECT 
  balance_decile,
  COUNT(*) AS total_clients,
  SUM(TARGET) AS total_defaults,
  ROUND(SUM(TARGET) * 1.0 / COUNT(*), 4) AS default_rate
FROM balance_bins
GROUP BY balance_decile
ORDER BY balance_decile;
"""

cursor.execute(query)
plan = cursor.fetchall()
for row in plan:
    print(row[0])

Sort  (cost=292098.21..292098.71 rows=200 width=52) (actual time=1248.635..1248.659 rows=10 loops=1)
  Sort Key: (ntile(10) OVER (?))
  Sort Method: quicksort  Memory: 25kB
  ->  HashAggregate  (cost=292086.06..292090.56 rows=200 width=52) (actual time=1248.621..1248.647 rows=10 loops=1)
        Group Key: ntile(10) OVER (?)
        Batches: 1  Memory Usage: 40kB
        ->  WindowAgg  (cost=281323.19..286704.62 rows=307511 width=44) (actual time=1203.517..1231.254 rows=307511 loops=1)
              ->  Sort  (cost=281323.18..282091.95 rows=307511 width=36) (actual time=1173.008..1187.351 rows=307511 loops=1)
                    Sort Key: c.avg_balance
                    Sort Method: external merge  Disk: 5200kB
                    ->  Hash Left Join  (cost=219424.28..244882.62 rows=307511 width=36) (actual time=1087.748..1145.005 rows=307511 loops=1)
                          Hash Cond: (a.sk_id_curr = c.sk_id_curr)
                          ->  Seq Scan on application_train a  (cost

**Análisis** 

El plan de ejecución para esta consulta muestra una operación compleja que involucra un Hash Left Join entre application_train y una subconsulta agregada de credit_card_balance. Se observa que ambas tablas son leídas completamente mediante escaneos secuenciales y que se realizan múltiples operaciones de ordenamiento (Sort) y agregación (HashAggregate, GroupAggregate) sobre grandes volúmenes de datos.

El cálculo de los deciles (NTILE(10) OVER (ORDER BY c.avg_balance)) requiere ordenar todos los datos por el saldo promedio, lo que genera un uso intensivo de disco y memoria, especialmente en los pasos de ordenamiento y agregación. El tiempo total de ejecución es elevado (alrededor de 1.4 segundos), lo que indica que la consulta es costosa en recursos y puede escalar mal si las tablas crecen.

Para optimizar esta consulta, se podrían considerar las siguientes acciones:

* Considerar la creación de índices sobre las columnas `SK_ID_CURR` en ambas tablas para acelerar los joins y sobre `avg_balance` si se realizan consultas similares frecuentemente.
  
* Evaluar la posibilidad de materializar la subconsulta de saldos promedio (credit_balance) si se utiliza en varias consultas, para evitar recalcularla cada vez.

* Monitorear el uso de disco y memoria, ya que los ordenamientos externos y las agregaciones sobre grandes volúmenes pueden afectar el rendimiento general del sistema.

* Revisar periódicamente el plan de ejecución tras crear los índices o modificar el volumen de datos, para asegurar que el motor de base de datos utiliza los caminos más eficientes.

#### 📌 `Query Pregunta 5.`

In [7]:
query = """
EXPLAIN ANALYZE
WITH credit_usage AS (
  SELECT 
    SK_ID_CURR,
    AVG(AMT_BALANCE / NULLIF(AMT_CREDIT_LIMIT_ACTUAL, 0)) AS avg_utilization
  FROM credit_card_balance
  WHERE AMT_CREDIT_LIMIT_ACTUAL > 0
  GROUP BY SK_ID_CURR
),
usage_bins AS (
  SELECT 
    a.SK_ID_CURR,
    a.TARGET,
    NTILE(10) OVER (ORDER BY c.avg_utilization) AS utilization_decile
  FROM application_train a
  LEFT JOIN credit_usage c ON a.SK_ID_CURR = c.SK_ID_CURR
)
SELECT 
  utilization_decile,
  COUNT(*) AS total_clients,
  SUM(TARGET) AS total_defaults,
  ROUND(SUM(TARGET) * 1.0 / COUNT(*), 4) AS default_rate
FROM usage_bins
GROUP BY utilization_decile
ORDER BY utilization_decile;
"""

cursor.execute(query)
plan = cursor.fetchall()
for row in plan:
    print(row[0])

Sort  (cost=278863.50..278864.00 rows=200 width=52) (actual time=1246.512..1246.532 rows=10 loops=1)
  Sort Key: (ntile(10) OVER (?))
  Sort Method: quicksort  Memory: 25kB
  ->  HashAggregate  (cost=278851.35..278855.85 rows=200 width=52) (actual time=1246.502..1246.524 rows=10 loops=1)
        Group Key: ntile(10) OVER (?)
        Batches: 1  Memory Usage: 40kB
        ->  WindowAgg  (cost=268088.49..273469.91 rows=307511 width=44) (actual time=1209.411..1230.817 rows=307511 loops=1)
              ->  Sort  (cost=268088.47..268857.25 rows=307511 width=36) (actual time=1178.436..1193.197 rows=307511 loops=1)
                    Sort Key: c.avg_utilization
                    Sort Method: external merge  Disk: 5264kB
                    ->  Hash Left Join  (cost=206189.57..231647.91 rows=307511 width=36) (actual time=1093.525..1150.590 rows=307511 loops=1)
                          Hash Cond: (a.sk_id_curr = c.sk_id_curr)
                          ->  Seq Scan on application_train a  (

**Análisis**

El plan de ejecución para esta consulta muestra una operación compleja que involucra un Hash Left Join entre application_train y una subconsulta agregada de credit_card_balance (donde se calcula la utilización promedio de la línea de crédito). Ambas tablas son leídas completamente mediante escaneos secuenciales y se realizan múltiples operaciones de ordenamiento (Sort) y agregación (HashAggregate, GroupAggregate) sobre grandes volúmenes de datos.

El cálculo de los deciles (NTILE(10) OVER (ORDER BY c.avg_utilization)) requiere ordenar todos los datos por la utilización promedio, lo que genera un uso intensivo de disco y memoria, especialmente en los pasos de ordenamiento y agregación. El tiempo total de ejecución es elevado (alrededor de 1.5 segundos), lo que indica que la consulta es costosa en recursos y puede escalar mal si las tablas crecen.

Para optimizar esta consulta, se podrían considerar las siguientes acciones:

* Considerar la creación de índices sobre las columnas `SK_ID_CURR` en ambas tablas para acelerar los joins y sobre avg_utilization si se realizan consultas similares frecuentemente.

* Evaluar la posibilidad de materializar la subconsulta de utilización promedio (credit_usage) si se utiliza en varias consultas, para evitar recalcularla cada vez.

* Monitorear el uso de disco y memoria, ya que los ordenamientos externos y las agregaciones sobre grandes volúmenes pueden afectar el rendimiento general del sistema.

* Revisar periódicamente el plan de ejecución tras crear los índices o modificar el volumen de datos, para asegurar que el motor de base de datos utiliza los caminos más eficientes.

#### 📌 `Query Pregunta 6.`

In [8]:
query = """
EXPLAIN ANALYZE
WITH refused_clients AS (
  SELECT 
    SK_ID_CURR,
    COUNT(*) AS refused_count
  FROM previous_application
  WHERE NAME_CONTRACT_STATUS = 'Refused'
  GROUP BY SK_ID_CURR
),
clients_flagged AS (
  SELECT 
    a.SK_ID_CURR,
    a.TARGET,
    CASE WHEN r.refused_count IS NOT NULL THEN 1 ELSE 0 END AS had_refusals
  FROM application_train a
  LEFT JOIN refused_clients r ON a.SK_ID_CURR = r.SK_ID_CURR
)
SELECT 
  had_refusals,
  COUNT(*) AS total_clients,
  SUM(TARGET) AS total_defaults,
  ROUND(SUM(TARGET) * 1.0 / COUNT(*), 4) AS default_rate
FROM clients_flagged
GROUP BY had_refusals;
"""

cursor.execute(query)
plan = cursor.fetchall()
for row in plan:
    print(row[0])

Finalize GroupAggregate  (cost=126576.23..126630.40 rows=200 width=52) (actual time=422.874..423.527 rows=2 loops=1)
  Group Key: (CASE WHEN (r.refused_count IS NOT NULL) THEN 1 ELSE 0 END)
  ->  Gather Merge  (cost=126576.23..126622.90 rows=400 width=20) (actual time=422.865..423.517 rows=6 loops=1)
        Workers Planned: 2
        Workers Launched: 2
        ->  Sort  (cost=125576.21..125576.71 rows=200 width=20) (actual time=417.379..417.380 rows=2 loops=3)
              Sort Key: (CASE WHEN (r.refused_count IS NOT NULL) THEN 1 ELSE 0 END)
              Sort Method: quicksort  Memory: 25kB
              Worker 0:  Sort Method: quicksort  Memory: 25kB
              Worker 1:  Sort Method: quicksort  Memory: 25kB
              ->  Partial HashAggregate  (cost=125566.56..125568.56 rows=200 width=20) (actual time=417.364..417.366 rows=2 loops=3)
                    Group Key: CASE WHEN (r.refused_count IS NOT NULL) THEN 1 ELSE 0 END
                    Batches: 1  Memory Usage: 40kB
 

**Análisis**

El plan de ejecución para esta consulta muestra una operación que involucra un Hash Left Join entre application_train y una subconsulta agregada de previous_application (donde se cuentan los rechazos por cliente). Ambas tablas son leídas completamente mediante escaneos secuenciales y se utilizan agregaciones (HashAggregate, GroupAggregate) y ordenamientos (Sort) en paralelo para procesar los datos.

El uso de procesamiento paralelo permite distribuir la carga entre varios trabajadores, lo que mejora el tiempo de respuesta (alrededor de 590 ms en total). Sin embargo, el acceso a los datos sigue siendo mediante escaneo secuencial, y la subconsulta sobre previous_application requiere filtrar y agrupar una gran cantidad de filas, lo que puede ser costoso si la tabla crece.

Para optimizar esta consulta, se podrían considerar las siguientes acciones:

* Considerar la creación de índices sobre las columnas `SK_ID_CURR` en ambas tablas para acelerar los joins y sobre `NAME_CONTRACT_STATUS` en previous_application para optimizar el filtrado por rechazos.

* Monitorear el uso de recursos (memoria y disco) durante las agregaciones y joins, especialmente si el volumen de datos aumenta.

* Revisar periódicamente el plan de ejecución tras crear los índices o modificar el volumen de datos, para asegurar que el motor de base de datos utiliza los caminos más eficientes.
Si la consulta se utiliza frecuentemente, evaluar la posibilidad de materializar la subconsulta de rechazos por cliente para evitar recalcularla cada vez.

#### 📌 `Query Pregunta 7.`

In [9]:
query = """
EXPLAIN ANALYZE
WITH labeled_clients AS (
  SELECT 
    SK_ID_CURR,
    TARGET,
    AMT_INCOME_TOTAL,
    CASE 
      WHEN NAME_TYPE_SUITE = 'Family' AND CNT_CHILDREN > 0 THEN 'With_Family_and_Children'
      WHEN NAME_TYPE_SUITE = 'Unaccompanied' AND CNT_CHILDREN = 0 THEN 'Alone_No_Children'
      ELSE 'Other'
    END AS group_type
  FROM application_train
),
income_deciles AS (
  SELECT *,
    NTILE(10) OVER (ORDER BY AMT_INCOME_TOTAL) AS income_decile
  FROM labeled_clients
)
SELECT 
  group_type,
  income_decile,
  COUNT(*) AS total_clients,
  SUM(TARGET) AS total_defaults,
  ROUND(SUM(TARGET) * 1.0 / COUNT(*), 4) AS default_rate
FROM income_deciles
WHERE group_type IN ('With_Family_and_Children', 'Alone_No_Children')
GROUP BY group_type, income_decile
ORDER BY group_type, income_decile;
"""

cursor.execute(query)
plan = cursor.fetchall()
for row in plan:
    print(row[0])

GroupAggregate  (cost=72520.25..72624.83 rows=2940 width=84) (actual time=224.526..248.724 rows=20 loops=1)
  Group Key: income_deciles.group_type, income_deciles.income_decile
  ->  Sort  (cost=72520.25..72527.93 rows=3075 width=40) (actual time=222.185..237.601 rows=187698 loops=1)
        Sort Key: income_deciles.group_type, income_deciles.income_decile
        Sort Method: external merge  Disk: 7088kB
        ->  Subquery Scan on income_deciles  (cost=60041.68..72342.11 rows=3075 width=40) (actual time=136.859..193.337 rows=187698 loops=1)
              Filter: (income_deciles.group_type = ANY ('{With_Family_and_Children,Alone_No_Children}'::text[]))
              Rows Removed by Filter: 119813
              ->  WindowAgg  (cost=60041.68..68498.22 rows=307511 width=50) (actual time=136.857..177.116 rows=307511 loops=1)
                    ->  Sort  (cost=60041.67..60810.44 rows=307511 width=27) (actual time=89.092..115.464 rows=307511 loops=1)
                          Sort Key: ap

**Análisis**

El plan de ejecución para esta consulta muestra que se realiza un escaneo secuencial completo sobre la tabla application_train, seguido de un ordenamiento externo (Sort) por el campo `AMT_INCOME_TOTAL` para calcular los deciles mediante una función de ventana (WindowAgg). Posteriormente, se filtran los grupos de interés y se realiza una agregación final (GroupAggregate) por tipo de grupo e income decile.

El uso de ordenamientos externos y funciones de ventana sobre grandes volúmenes de datos genera un uso intensivo de disco y memoria (por ejemplo, el sort utiliza más de 12 MB en disco). El tiempo total de ejecución es moderado (alrededor de 420 ms), pero puede aumentar si la tabla crece.

Para optimizar esta consulta, se podrían considerar las siguientes acciones:

* Considerar la creación de un índice sobre `AMT_INCOME_TOTAL` para acelerar el ordenamiento requerido por la función de ventana.

* Monitorear el uso de disco y memoria, ya que los ordenamientos externos pueden impactar el rendimiento general si el volumen de datos aumenta.

* Si la consulta se utiliza frecuentemente, evaluar la posibilidad de materializar los resultados intermedios (por ejemplo, los deciles) para evitar recalcularlos cada vez.

* Revisar periódicamente el plan de ejecución tras crear índices o cambios en el volumen de datos, para asegurar que el motor de base de datos utiliza los caminos más eficientes.

#### 📌 `Query Pregunta 8.`

In [10]:
query = """
EXPLAIN ANALYZE
WITH categorized_clients AS (
  SELECT 
    SK_ID_CURR,
    TARGET,
    AMT_INCOME_TOTAL,
    CASE 
      WHEN NAME_HOUSING_TYPE IN ('Rented apartment', 'Municipal apartment') 
        AND NAME_EDUCATION_TYPE IN ('Secondary / secondary special', 'Lower secondary') 
        THEN 'Low_Edu_Rent'
      WHEN NAME_HOUSING_TYPE = 'House / apartment' 
        AND NAME_EDUCATION_TYPE = 'Higher education'
        THEN 'High_Edu_Owner'
      ELSE 'Other'
    END AS profile_type
  FROM application_train
),
income_deciles AS (
  SELECT *,
    NTILE(10) OVER (ORDER BY AMT_INCOME_TOTAL) AS income_decile
  FROM categorized_clients
)
SELECT 
  profile_type,
  income_decile,
  COUNT(*) AS total_clients,
  SUM(TARGET) AS total_defaults,
  ROUND(SUM(TARGET) * 1.0 / COUNT(*), 4) AS default_rate
FROM income_deciles
WHERE profile_type IN ('Low_Edu_Rent', 'High_Edu_Owner')
GROUP BY profile_type, income_decile
ORDER BY profile_type, income_decile;
"""

cursor.execute(query)
plan = cursor.fetchall()
for row in plan:
    print(row[0])

GroupAggregate  (cost=75673.75..75778.33 rows=2940 width=84) (actual time=232.906..241.099 rows=20 loops=1)
  Group Key: income_deciles.profile_type, income_deciles.income_decile
  ->  Sort  (cost=75673.75..75681.43 rows=3075 width=40) (actual time=232.589..236.742 rows=79549 loops=1)
        Sort Key: income_deciles.profile_type, income_deciles.income_decile
        Sort Method: external merge  Disk: 2656kB
        ->  Subquery Scan on income_deciles  (cost=63195.18..75495.61 rows=3075 width=40) (actual time=158.514..218.325 rows=79549 loops=1)
              Filter: (income_deciles.profile_type = ANY ('{Low_Edu_Rent,High_Edu_Owner}'::text[]))
              Rows Removed by Filter: 227962
              ->  WindowAgg  (cost=63195.18..71651.72 rows=307511 width=50) (actual time=158.504..205.473 rows=307511 loops=1)
                    ->  Sort  (cost=63195.17..63963.94 rows=307511 width=53) (actual time=103.409..131.824 rows=307511 loops=1)
                          Sort Key: application_

**Análisis**

El plan de ejecución para esta consulta muestra que se realiza un escaneo secuencial completo sobre la tabla application_train, seguido de un ordenamiento externo (Sort) por el campo `AMT_INCOME_TOTAL` para calcular los deciles mediante una función de ventana (WindowAgg). Luego, se filtran los perfiles de interés y se realiza una agregación final (GroupAggregate) por tipo de perfil e income decile.

El uso de ordenamientos externos y funciones de ventana sobre grandes volúmenes de datos genera un uso intensivo de disco y memoria (por ejemplo, el sort utiliza cerca de 20 MB en disco). El tiempo total de ejecución es moderado (alrededor de 350 ms), pero puede aumentar si la tabla crece.

Para optimizar esta consulta, se podrían considerar las siguientes acciones:

* Considerar la creación de un índice sobre `AMT_INCOME_TOTAL` para acelerar el ordenamiento requerido por la función de ventana.

* Monitorear el uso de disco y memoria, ya que los ordenamientos externos pueden impactar el rendimiento general si el volumen de datos aumenta.

* Si la consulta se utiliza frecuentemente, evaluar la posibilidad de materializar los resultados intermedios (por ejemplo, los deciles) para evitar recalcularlos cada vez.

* Revisar periódicamente el plan de ejecución tras crear índices o cambios en el volumen de datos, para asegurar que el motor de base de datos utiliza los caminos más eficientes.



#### `Creación de Index`

In [12]:
index_commands = [
    "CREATE INDEX IF NOT EXISTS idx_application_train_sk_id_curr ON application_train(SK_ID_CURR);",
    "CREATE INDEX IF NOT EXISTS idx_credit_card_balance_sk_id_curr ON credit_card_balance(SK_ID_CURR);",
    "CREATE INDEX IF NOT EXISTS idx_bureau_sk_id_curr ON bureau(SK_ID_CURR);",
    "CREATE INDEX IF NOT EXISTS idx_previous_application_sk_id_curr ON previous_application(SK_ID_CURR);",
    "CREATE INDEX IF NOT EXISTS idx_application_train_amt_income_total ON application_train(AMT_INCOME_TOTAL);",
    "CREATE INDEX IF NOT EXISTS idx_application_train_amt_credit ON application_train(AMT_CREDIT);",
    "CREATE INDEX IF NOT EXISTS idx_previous_application_name_contract_status ON previous_application(NAME_CONTRACT_STATUS);",
    "CREATE INDEX IF NOT EXISTS idx_application_train_name_contract_type ON application_train(NAME_CONTRACT_TYPE);",
    "CREATE INDEX IF NOT EXISTS idx_application_train_name_housing_type ON application_train(NAME_HOUSING_TYPE);",
    "CREATE INDEX IF NOT EXISTS idx_application_train_name_education_type ON application_train(NAME_EDUCATION_TYPE);"
]

for cmd in index_commands:
    cursor.execute(cmd)
conn.commit()

In [13]:
# Cierre conexión
cursor.close()
conn.close()